In [1]:
!pip install selenium pandas webdriver-manager

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


# Avec sauvegarde

In [3]:
import pandas as pd

df=pd.read_csv('3-airbnb_marrakech.csv')
df.head(10)

,titre,localisation,voyageurs,chambres,lits,salles_de_bain,prix_total,note,nb_avis,amenities,latitude,longitude,lien
0,Appartement de luxe dans le centre de Marrakech,Appartement ⋅ Marrakech,4.0,3.0,3.0,2.0,234 €,NaN,NaN,NaN,31.66060,-8.03090,https://www.airbnb.fr/rooms/160366998127511560...
1,Appartement confort Guéliz– 2 ch • Piscine • F...,Appartement ⋅ Marrakech,NaN,NaN,NaN,NaN,368 €,"4,88",103.0,Autres : Oreillers et couvertures supplémentai...,31.62860,-8.01980,https://www.airbnb.fr/rooms/136403005912834921...
2,Luxe Apt Sécurisé et accès facile aux commodités,Appartement ⋅ Marrakech,NaN,NaN,NaN,NaN,239 €,"5,0",7.0,Autres : Caméras de surveillance extérieures p...,31.64620,-8.01690,https://www.airbnb.fr/rooms/159340552046648771...
3,Appartement de luxe 3 chambres/terrasses centr...,Appartement ⋅ Marrakech,9.0,3.0,4.0,2.0,363 €,NaN,NaN,Autres : Oreillers et couvertures supplémentai...,31.63090,-8.01400,https://www.airbnb.fr/rooms/159722153509930820...
4,"Appartement moderne, cosy et proche Aéroport",Appartement ⋅ Marrakech,5.0,2.0,3.0,1.0,443 €,"5,0",24.0,Autres : Oreillers et couvertures supplémentai...,31.58970,-8.01740,https://www.airbnb.fr/rooms/157385455088392122...
5,"Studio style hôtel | Fibre rapide, Parking",Appartement ⋅ Marrakech,3.0,1.0,1.0,1.0,195 €,"5,0",3.0,Autres : Oreillers et couvertures supplémentai...,31.66370,-8.01530,https://www.airbnb.fr/rooms/160437565429377207...
6,Appartement luxe avec piscine-5 min centre ville,Appartement ⋅ Marrakech,6.0,2.0,2.0,2.0,425 €,NaN,NaN,Autres : Vaisselle et couverts | Autres : Sèch...,31.61131,-7.98851,https://www.airbnb.fr/rooms/159972890806620423...
7,Douiria Bleu Azur au cœur de la Médina,Riad ⋅ Marrakech,4.0,2.0,2.0,1.0,298 €,"5,0",3.0,Autres : Douche extérieure | Autres : Oreiller...,31.62521,-7.98267,https://www.airbnb.fr/rooms/160166626932313672...
8,Flextay VILLA | 4BR | Piscine | 12 min centre,Hébergement ⋅ Marrakech,NaN,NaN,NaN,NaN,971 €,NaN,NaN,Autres : Douche extérieure | Autres : Four Sim...,31.66410,-8.06980,https://www.airbnb.fr/rooms/159503087090712808...
9,"NOUVEAU Appartement confortable 2BR / Gueliz, ...",Appartement ⋅ Marrakech,4.0,2.0,3.0,2.0,448 €,"5,0",7.0,Autres : Oreillers et couvertures supplémentai...,31.63651,-8.01041,https://www.airbnb.fr/rooms/158694343527837216...


In [4]:
df.shape

(270, 13)

In [1]:
# 

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import re
import random
import csv
import os

from selenium.common.exceptions import NoSuchElementException

# ---------------------------
# LISTE DE LIENS À SCRAPER
# ---------------------------
urls = [
    "https://www.airbnb.fr/s/Marrakech/homes",
    "https://www.airbnb.fr/s/Medina--Marrakech/homes",
    "https://www.airbnb.fr/s/Gueliz--Marrakech/homes",
    "https://www.airbnb.fr/s/Hivernage--Marrakech/homes",
    "https://www.airbnb.fr/s/Mellah--Marrakech/homes",
    "https://www.airbnb.fr/s/Palmeraie--Marrakech/homes",
    "https://www.airbnb.fr/s/Agdal--Marrakech/homes",
    "https://www.airbnb.fr/s/Sidi-Ghanem--Marrakech/homes",
    "https://www.airbnb.fr/s/Targa--Marrakech/homes",
    "https://www.airbnb.fr/s/Route-de-Fes--Marrakech/homes",
    "https://www.airbnb.fr/s/Route-de-Ouarzazate--Marrakech/homes",
    "https://www.airbnb.fr/s/Amelkis--Marrakech/homes",
    "https://www.airbnb.fr/s/Menara--Marrakech/homes"
]



OUTPUT_FILE = "4-airbnb_marrakech.csv"

# ---------------------------
# CONFIG CHROME
# ---------------------------
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--lang=fr")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 20)

# ---------------------------
# INIT CSV (header une seule fois)
# ---------------------------
HEADERS = [
    "titre","localisation","voyageurs","chambres","lits","salles_de_bain",
    "prix_total","nb_nuits","note","nb_avis","amenities","latitude","longitude","host_verified","lien"
]


if not os.path.exists(OUTPUT_FILE):
    with open(OUTPUT_FILE, "w", newline="", encoding="utf-8-sig") as f:
        writer = csv.DictWriter(f, fieldnames=HEADERS)
        writer.writeheader()

def save_row(row):
    with open(OUTPUT_FILE, "a", newline="", encoding="utf-8-sig") as f:
        writer = csv.DictWriter(f, fieldnames=HEADERS)
        writer.writerow(row)

# ---------------------------
# SCROLL INTELLIGENT
# ---------------------------
def smart_scroll(n=15):
    for _ in range(n):
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        time.sleep(random.uniform(2.5,4))

# ---------------------------
# POPUP TRADUCTION
# ---------------------------
def close_translation_popup(driver, wait):
    try:
        popup = wait.until(EC.presence_of_element_located(
            (By.XPATH, '//header[@data-testid="translation-announce-modal"]')
        ))
        popup.find_element(By.XPATH, './/button').click()
        time.sleep(1)
    except:
        pass

# ---------------------------
# FONCTIONS DÉTAIL
# ---------------------------
def extract_room_specs(driver, wait):
    try:
        wait.until(EC.presence_of_element_located(
            (By.XPATH, '//span[contains(text(),"voyageur") or contains(text(),"chambre")]')
        ))
        full_text = driver.find_element(By.XPATH, '//main').text.lower()
        voyageurs = re.search(r'(\d+)\s*voyageur', full_text)
        chambres = re.search(r'(\d+)\s*chambre', full_text)
        lits = re.search(r'(\d+)\s*lit', full_text)
        sdb = re.search(r'(\d+(?:[.,]\d+)?)\s*salle', full_text)
        return {
            "voyageurs": float(voyageurs.group(1)) if voyageurs else None,
            "chambres": float(chambres.group(1)) if chambres else None,
            "lits": float(lits.group(1)) if lits else None,
            "salles_de_bain": float(sdb.group(1)) if sdb else None
        }
    except:
        return {"voyageurs":None,"chambres":None,"lits":None,"salles_de_bain":None}

# ---------------------------
# AMENITIES
# ---------------------------
def click_show_amenities_button(driver, wait):
    try:
        title_xpath = '//h2[contains(text(), "Ce que propose ce logement")]'
        button_xpath = (
            f'{title_xpath}/ancestor-or-self::div[1]'
            '//following::button[.//span[contains(text(), "Afficher")]][1]'
        )
        btn = wait.until(EC.element_to_be_clickable((By.XPATH, button_xpath)))
        driver.execute_script("arguments[0].scrollIntoView({block:'center'});", btn)
        time.sleep(0.6)
        driver.execute_script("arguments[0].click();", btn)
        time.sleep(1.4)
        return True
    except:
        return False

def extract_amenities_complete(driver, wait):
    amenities = []
    click_show_amenities_button(driver, wait)
    try:
        wait.until(EC.presence_of_element_located(
            (By.XPATH, '//div[contains(@id,"-row-title")]')
        ))
        rows = driver.find_elements(By.XPATH, '//div[contains(@id,"-row-title")]')
        current_cat = "Autres"
        for row in rows:
            text = row.text.strip().replace('\xa0', ' ')
            if not text:
                continue
            try:
                cat_h2 = row.find_element(By.XPATH, 'ancestor::*[h2][1]//h2')
                current_cat = cat_h2.text.strip()
            except:
                pass
            amenities.append(f"{current_cat} : {text}")
    except:
        pass
    return list(set(amenities))

# ---------------------------
# COORDONNÉES
# ---------------------------
def extract_geo_coordinates(driver):
    try:
        html = driver.page_source
        lat = re.search(r'"lat":\s*(-?\d+\.\d+)', html)
        lng = re.search(r'"lng":\s*(-?\d+\.\d+)', html)
        if lat and lng:
            return float(lat.group(1)), float(lng.group(1))
        return None, None
    except:
        return None, None

# ---------------------------
# EXTRACTION DES DONNÉES HÔTE
# ---------------------------
def get_host_info(driver, wait):
    # Superhôte
    try:
        superhost = wait.until(EC.presence_of_element_located(
            (By.CSS_SELECTOR, "span.s1vpsye7")
        )).text
    except:
        superhost = ""

    # Nombre d'évaluations
    try:
        host_evals = driver.find_element(By.CSS_SELECTOR, "span.a8jt5op").text.strip()
    except:
        host_evals = ""

    # Note moyenne
    try:
        host_rating = driver.find_element(By.CSS_SELECTOR, "div.rz5w5y3").text.strip()
    except:
        host_rating = ""

    # Années en tant qu'hôte
    try:
        host_years = driver.find_element(By.XPATH, "//span[contains(text(),'an en tant')]").text.strip()
    except:
        host_years = ""

    # Identité vérifiée
    try:
        driver.find_element(By.CSS_SELECTOR, "svg[aria-label='Identité vérifiée']")
        host_verified = "Oui"
    except NoSuchElementException:
        host_verified = "Non"

    return host_evals, host_rating, host_years, host_verified, superhost


# ---------------------------
# SCRAPING PRINCIPAL
# ---------------------------
for url in urls:
    print(f"\n🔗 Lien actuel : {url}")
    driver.get(url)
    time.sleep(8)
    page = 1

    while True:
        print(f"\n📄 PAGE {page}")
        smart_scroll(15)

        try:
            cards = wait.until(EC.presence_of_all_elements_located(
                (By.XPATH, '//div[@itemprop="itemListElement"]')
            ))
        except:
            print("Aucune annonce trouvée sur cette page.")
            break

        print("Nombre d'annonces détectées :", len(cards))

        for i, card in enumerate(cards, start=1):
            print(f"Scraping annonce {i}/{len(cards)} (page {page})")

            try: title = card.find_element(By.XPATH,'.//span[@data-testid="listing-card-name"]').text
            except: title = None

            try: location = card.find_element(By.XPATH,'.//div[@data-testid="listing-card-title"]').text
            except: location = None

            try: prix = card.find_element(By.XPATH,'.//span[contains(text(),"€")]').text
            except: prix = None

            try:
                rating_block = card.find_element(By.XPATH,'.//span[contains(text(),"(")]').text
                note = re.search(r'([\d,.]+)', rating_block).group(1)
                avis = re.search(r'\((\d+)\)', rating_block).group(1)
            except:
                note, avis = None, None

            try:
                href = card.find_element(By.XPATH,'.//a[contains(@href,"/rooms/")]').get_attribute("href")
            except:
                href = None

            voyageurs = chambres2 = lits2 = sdb = None
            amenities = []
            latitude = longitude = None

            if href:
                driver.execute_script("window.open(arguments[0]);", href)
                driver.switch_to.window(driver.window_handles[1])

                close_translation_popup(driver, wait)

                specs = extract_room_specs(driver, wait)
                voyageurs = specs["voyageurs"]
                chambres2 = specs["chambres"]
                lits2 = specs["lits"]
                sdb = specs["salles_de_bain"]

                amenities = extract_amenities_complete(driver, wait)
                latitude, longitude = extract_geo_coordinates(driver)
                host_evals, host_rating, host_years, host_verified, superhost = get_host_info(driver, wait)

                # élément H2 contenant "nuits"
                try:
                    h2 = wait.until(
                        EC.presence_of_element_located(
                            (By.XPATH, "//h2[contains(text(),'nuits')]")
                        )
                    ).text
                    match = re.search(r"(\d+)", h2)  # <--- utiliser h2 ici
                    nb_nuits = int(match.group(1)) if match else None
                except:
                    nb_nuits = None

                driver.close()
                driver.switch_to.window(driver.window_handles[0])
                time.sleep(random.uniform(2,4))

            row = {
                "titre": title,
                "localisation": location,
                "voyageurs": voyageurs,
                "chambres": chambres2,
                "lits": lits2,
                "salles_de_bain": sdb,
                "prix_total": prix,
                "nb_nuits": nb_nuits,
                "note": note,
                "nb_avis": avis,
                "amenities": " | ".join(amenities),
                "latitude": latitude,
                "longitude": longitude,
                "host_verified": host_verified,
                "lien": href
            }

            save_row(row)
            print('data \n', row)
            print("✔ sauvegardé")

        try:
            next_btn = wait.until(EC.element_to_be_clickable((By.XPATH, '//a[@aria-label="Suivant"]')))
            driver.execute_script("arguments[0].click();", next_btn)
            time.sleep(8)
            page += 1
        except:
            print("\n✅ Toutes les pages ont été parcourues pour ce lien.")
            break

driver.quit()
print("\nScraping terminé pour tous les liens ✅")



🔗 Lien actuel : https://www.airbnb.fr/s/Marrakech/homes

📄 PAGE 1
Nombre d'annonces détectées : 18
Scraping annonce 1/18 (page 1)
data 
 {'titre': 'Studio charmant près de Gueliz à Marrakech', 'localisation': 'Appartement ⋅ Marrakech', 'voyageurs': 4.0, 'chambres': 1.0, 'lits': 1.0, 'salles_de_bain': 1.0, 'prix_total': '161 €', 'nb_nuits': 5, 'note': None, 'nb_avis': None, 'amenities': 'Autres : Lit pour bébé | Autres : Indisponible : Détecteur de monoxyde de carbone\nDétecteur de monoxyde de carbone | Autres : Machine à pain | Autres : Stores | Autres : Vaisselle et couverts | Autres : Plaque de cuisson | Autres : Sèche-linge | Autres : Oreillers et couvertures supplémentaires | Autres : Boîte à clé sécurisée | Autres : Chauffage | Autres : Wifi | Autres : Séjours longue durée autorisés | Autres : Indisponible : Caméras de surveillance extérieures présentes sur place\nCaméras de surveillance extérieures présentes sur place | Autres : Étendoir à linge | Autres : Congélateur | Autres :